
# Generating Movie Recommendations

### Unsupervised_Learning_ZM6

1. Noxolo Ngcobo
2. Mora Magakwe
3. Sandra Malope
4. Katleho Moketo
5. Shuaib Morris
6. Matthews Montle


## Introduction

Recommendation systems are becoming increasingly important in today’s extremely busy world. People are forever short on time due to the tasks they need to accomplish in the limited time they have. Therefore, the recommendation systems are very important as they assist in making the right choices.

The purpose of a recommendation system basically is to search for content that would be interesting to an individual. Moreover, it involves a number of factors to create personalised lists of useful and interesting content specific to each user/individual. Recommendation systems are Artificial Intelligence based algorithms that skim through all possible options and create a customized list of items that are interesting and relevant to an individual. These results are based on their profile, search/browsing history, what other people with similar traits/demographics are watching, and how likely are you to watch those movies. This is achieved through predictive modeling and heuristics with the data available.

With this context, EDSA is challenging us to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.


<div align="center" style="width: 900px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/kmoketo-dev/ZM6_Unsurpervised-Learning-Predict/main/intro.jpg"
     alt="Titanic"
     style="float: center; padding-bottom=0.5em"
     width=900px/>

</div>


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installing packages
Please download all relevant packages in. There is no terminal so you will pip install everything.

You can find a list of recommended install from the Intro to Recommender sysytem notebook.

In [1]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

## Reading in data

In [2]:
# df_sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/sample_submission.csv')
# df_movies = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/movies.csv')
# df_imdb = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/imdb_data.csv')
# df_genome_scores = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/genome_scores.csv')
# df_genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/genome_tags.csv')
# df_train = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/train.csv')
# df_test = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/test.csv')
# df_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/tags.csv')
# df_links = pd.read_csv('/kaggle/input/edsa-movie-recommendation-challenge/links.csv')

In [2]:
df_sample_submission = pd.read_csv('sample_submission.csv')
df_movies = pd.read_csv('movies.csv')
df_imdb = pd.read_csv('imdb_data.csv')
df_genome_scores = pd.read_csv('genome_scores.csv')
df_genome_tags = pd.read_csv('genome_tags.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_tags = pd.read_csv('tags.csv')
df_links = pd.read_csv('links.csv')

In [ ]:
df_movies.head()

In [ ]:
df_imdb.head()

In [ ]:
df_genome_scores.head()

In [ ]:
df_genome_tags.head()

In [ ]:
df_train.head()

In [ ]:
df_tags.head()

In [ ]:
df_links.head()

In [ ]:
df_test.head()

In [ ]:
df_movies.shape

In [ ]:
df_train.shape

In [ ]:
df_movies.describe()

In [ ]:
df_train.describe()

In [ ]:
dataset = pd.merge(df_train,df_movies,on = 'movieId')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.nunique()

In [ ]:
dataset.describe()

## EDA
Discovery phase and data understanding

In [ ]:
#average rating
a=dataset
a=a.groupby('title')['rating'].mean().head()
a

In [ ]:
#the rating of each movie
b=dataset.groupby('title')['rating'].count()
b.head()

In [ ]:
#making a new dataframe
new_record = pd.DataFrame()
new_record['Average_ratings']=a

In [ ]:
new_record['Count of total ratings'] = b
new_record.head()

In [ ]:
#plot graph of num of 
plt.figure(figsize=(10,9))

new_record['Count of total ratings'].hist(bins = 100)

In [ ]:
#plot graph of ratings column
plt.figure(figsize =(10,4))

new_record['Average_ratings'].hist(bins = 70)

In [ ]:
plt.figure(figsize=(8,6))
sns.jointplot(x='Average_ratings',  y='Count of total ratings',data=new_record,alpha=0.4)

In [ ]:
movies_df=pd.merge(df_train, df_movies,on='movieId',how='inner')
movies_df.head()

**Genres with highest rating.**

In [ ]:
genre_count=movies_df['genres'].value_counts().sort_values(ascending=False)
genre_count=pd.DataFrame(genre_count)
top_genre=genre_count[0:11]
top_genre

In [ ]:
# To find the number of times a user rated a movie
user_df = pd.DataFrame(
    df_train['userId'].value_counts()).reset_index()
user_df.rename(columns={'index':'userId','userId':'count'},
                  inplace=True)
user_df.head(10)

In [ ]:
plt.figure(figsize=(14,7))
data = df_train['userId'].value_counts().head(10)
ax = sns.barplot(x = data.index, y = data, order= data.index, palette='CMRmap', edgecolor="black")
plt.title(f'Top 10 Users by Number of Ratings', fontsize=14)
plt.xlabel('User ID')
plt.ylabel('Number of Ratings')
plt.show()

The above output shows that user the with id **72315** has the highest rating count of **12952**

In [ ]:
movieRating_Group = df_train['rating'].value_counts().sort_index().reset_index()
fig, ax = plt.subplots(figsize=(14,7))
sns.barplot(data=movieRating_Group, x='index', y='rating', palette="CMRmap", edgecolor="black", ax=ax)
ax.set_xlabel("Rating")
ax.set_ylabel('Number of Users')
ax.set_yticklabels(['{:,}'.format(int(x)) for x in ax.get_yticks().tolist()])
total = float(movieRating_Group['rating'].sum())
plt.title('Number of Users Per Rating', fontsize=14)
plt.show()

**Most common Genres**

In [ ]:
# Create dataframe containing only the movieId and genres
movies_genres = pd.DataFrame(df_movies[['movieId', 'genres']],
                             columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-genre combination is in a seperate row
movies_genres = pd.DataFrame([(tup.movieId, d) for tup in movies_genres.itertuples() for d in tup.genres],
                             columns=['movieId', 'genres'])

movies_genres.head()

In [ ]:
# Plot the genres from most common to least common
plot = plt.figure(figsize=(15, 10))
plt.title('Most common genres\n', fontsize=20)
sns.countplot(y="genres", data=movies_genres,
              order=movies_genres['genres'].value_counts(ascending=False).index)
plt.show()

In [ ]:
dates = []
for title in movies_df['title']:
    if title[-1] == " ":
        year = title[-6: -2]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)
    else:
        year = title[-5: -1]
        try:
            dates.append(int(year))
        except:
            dates.append(9999)

movies_df['Year'] = dates

**Years in which movies were released**

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
ax = sns.countplot(y="Year", data=movies_df, palette="Set2", order=movies_df['Year'].value_counts().index[0:15])

So, **1995** was the year when most of the movies were released.

In [ ]:
dataset.groupby('title')['rating'].count().sort_values(ascending=False).head()

In [ ]:
ratings = pd.DataFrame(dataset.groupby('title')['rating'].mean())
ratings.head()

In [ ]:
ratings['num of ratings'] = pd.DataFrame(dataset.groupby('title')['rating'].count())
ratings.head()

In [ ]:
plt.figure(figsize=(10,4))
ratings['num of ratings'].hist(bins=2)

In [ ]:
plt.figure(figsize=(10,4))
ratings['rating'].hist(bins=70)

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha=0.5)

In [ ]:
years = []

for title in df_movies['title']:
    yearpub_subset = title[-5:-1]
    try: years.append(int(yearpub_subset))
    except: years.append(9999)
        
df_movies['yearpub'] = years
print(len(df_movies[df_movies['yearpub'] == 9999]))

In [ ]:
def make_histogram(dataset, attribute, bins=25, bar_color='#3498db', edge_color='#2980b9', title='Title', xlab='X', ylab='Y', sort_index=False):
    if attribute == 'yearpub':
        dataset = dataset[dataset['yearpub'] != 9999]
        
    fig, ax = plt.subplots(figsize=(14, 7))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_title(title, fontsize=24, pad=20)
    ax.set_xlabel(xlab, fontsize=16, labelpad=20)
    ax.set_ylabel(ylab, fontsize=16, labelpad=20)
    
    plt.hist(dataset[attribute], bins=bins, color=bar_color, ec=edge_color, linewidth=2)
    
    plt.xticks(rotation=45)
    
    
make_histogram(df_movies, 'yearpub', title='Movies per year distribution', xlab='year', ylab='Counts')

In [ ]:
ratings_df = pd.DataFrame()
ratings_df['Mean_Rating'] = dataset.groupby('title')['rating'].mean().values
ratings_df['Num_Ratings'] = dataset.groupby('title')['rating'].count().values


fig, ax = plt.subplots(figsize=(14, 7))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_title('Rating vs. Number of Ratings', fontsize=24, pad=20)
ax.set_xlabel('Rating', fontsize=16, labelpad=20)
ax.set_ylabel('Number of Ratings', fontsize=16, labelpad=20)

plt.scatter(ratings_df['Mean_Rating'], ratings_df['Num_Ratings'], alpha=0.5)

## Data Preparation and cleaning

In [20]:
df_train['rating'].value_counts()

4.0    2652977
3.0    1959759
5.0    1445230
3.5    1270642
4.5     880516
2.0     656821
2.5     505578
1.0     311213
1.5     159731
0.5     157571
Name: rating, dtype: int64

In [8]:
df_train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [22]:

users = df_train.sample(n = 10000000, replace = False)
len(users)

10000000

In [23]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(users[['userId', 'movieId', 'rating']], reader)

In [36]:
svd = SVD(verbose=True, n_epochs=12)
cross_validate(svd, data, measures=['RMSE'], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Evaluating RMSE of algorithm SVD on 4 split(s).

                  Fold 1  Fold 

{'test_rmse': array([0.85187862, 0.85192456, 0.8520577 , 0.85285586]),
 'fit_time': (335.9232130050659,
  361.2407441139221,
  344.3270184993744,
  396.21927762031555),
 'test_time': (32.16224217414856,
  32.692840337753296,
  34.473952531814575,
  32.94402766227722)}

# Generate your outputs here

Prepare Submission File
We make submissions in CSV files. Your submissions usually have two columns: an ID column and a prediction column. The ID field comes from the test data (keeping whatever name the ID field had in that data, which for the data is the string 'Id'). The prediction column will use the name of the target field.

We will create a DataFrame with this data, and then use the dataframe's to_csv method to write our submission file. Explicitly include the argument index=False to prevent pandas from adding another column in our csv file.

In [26]:
pred_svd_hyper = [svd.predict(row.userId,
                                         row.movieId) for idx,row in df_test.iterrows()]

# Converting the predictions to a dataframe
test_pred_svd_hyper = pd.DataFrame(pred_svd_hyper)


### Make Submission
Hit the blue Publish button at the top of your notebook screen. It will take some time for your kernel to run. When it has finished your navigation bar at the top of the screen will have a tab for Output. This only shows up if you have written an output file (like we did in the Prepare Submission File step).

Example below of how the output would look once published

In [27]:
# Rename the fields in the prediction dataframe
test_pred_svd_hyper.drop(['r_ui', 'details'], axis=1, inplace=True)
test_pred_svd_hyper = test_pred_svd_hyper.rename(columns={'uid':'userId',
                                                          'iid':'movieId',
                                                          'est':'rating'})
test_pred_svd_hyper.head()

,userId,movieId,rating
0,1,2011,3.633416
1,1,4144,4.159102
2,1,5767,3.567240
3,1,6711,3.770057
4,1,7318,3.072327


In [28]:
# Concatenate each userId and movieId into a single Id column for submission
test_pred_svd_hyper['Id'] =  test_pred_svd_hyper['userId'].astype(str).str.zfill(1) + '_' + test_pred_svd_hyper['movieId'].astype(str).str.zfill(1)

In [29]:
svd_hyper_predictions = test_pred_svd_hyper[['Id','rating']]
svd_hyper_predictions.head()

,Id,rating
0,1_2011,3.633416
1,1_4144,4.159102
2,1_5767,3.567240
3,1_6711,3.770057
4,1_7318,3.072327


In [30]:
svd_hyper_predictions.to_csv('submission4.csv', index=False)